# Variational Autoencoders

In this notebook, you'll convert a regular autoencoder into a VAE and see the generative payoff.

**What you'll do:**
- Implement the reparameterization trick and verify gradients flow through sampling
- Convert an autoencoder to a VAE by changing the encoder to output mu + log_var
- Implement the VAE loss function (reconstruction + KL divergence) and train the model
- Compare the VAE's latent space to the autoencoder's — see how KL regularization fills the gaps
- Generate new Fashion-MNIST items by sampling from the latent space

**For each exercise, PREDICT the output before running the cell.** Wrong predictions are more valuable than correct ones — they reveal gaps in your mental model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# Reproducible results
torch.manual_seed(42)
np.random.seed(42)

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 4]

# Device setup — use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Download Fashion-MNIST
transform = transforms.ToTensor()
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

CLASS_NAMES = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"Training set: {len(train_dataset)} images")
print(f"Test set: {len(test_dataset)} images")

## Shared Helpers

A regular autoencoder for comparison. This is the same architecture from the Autoencoders notebook — encoder compresses to a single point in latent space, decoder reconstructs.

In [ ]:
LATENT_DIM = 2  # Using 2D so we can visualize the latent space

class Autoencoder(nn.Module):
    """Regular autoencoder — encodes each image to a single point."""
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32 * 7 * 7),
            nn.ReLU(),
            nn.Unflatten(1, (32, 7, 7)),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


def train_autoencoder(model, train_loader, num_epochs=10, lr=1e-3):
    """Train a regular autoencoder with MSE reconstruction loss."""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, _ in train_loader:
            images = images.to(device)
            recon = model(images)
            loss = F.mse_loss(recon, images, reduction='sum') / images.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    return losses


# Train the regular autoencoder (we'll compare to the VAE later)
print("Training regular autoencoder...")
ae_model = Autoencoder(LATENT_DIM).to(device)
ae_losses = train_autoencoder(ae_model, train_loader, num_epochs=10)
print("Done.")

---

## Exercise 1: The Reparameterization Trick (Guided)

The VAE encodes each image to a distribution (mu + sigma), then **samples** z from that distribution during training. But sampling is random — how do gradients flow backward through randomness?

The trick: instead of sampling z directly from N(mu, sigma^2), we sample epsilon from N(0, 1) and compute:

$$z = \mu + \sigma \cdot \epsilon$$

Now z is a deterministic function of mu and sigma (the learnable parts). The randomness is isolated in epsilon, which is just input noise. Gradients flow through mu and sigma normally.

**Before running, predict:** If we sample z directly from N(mu, sigma^2), will `mu.grad` be None or have a value? What about with the reparameterization trick?

In [ ]:
# Attempt 1: Sample z directly (the naive way)
mu = torch.tensor([1.0, -0.5], requires_grad=True)
logvar = torch.tensor([0.0, 0.0], requires_grad=True)  # log(1.0) = 0, so sigma = 1

# This creates a Normal distribution and samples from it
# But torch.distributions sampling breaks the gradient path
sigma = torch.exp(0.5 * logvar)
dist = torch.distributions.Normal(mu, sigma)
z_direct = dist.sample()  # No gradient connection!

# Try to backprop through it
loss_direct = z_direct.sum()
loss_direct.backward()

print("=== Direct sampling (naive) ===")
print(f"z = {z_direct.detach().numpy()}")
print(f"mu.grad = {mu.grad}")
print(f"Gradient flows to mu? {mu.grad is not None and mu.grad.abs().sum() > 0}")
print()

# Reset gradients
mu = torch.tensor([1.0, -0.5], requires_grad=True)
logvar = torch.tensor([0.0, 0.0], requires_grad=True)

# Attempt 2: Reparameterization trick
sigma = torch.exp(0.5 * logvar)
eps = torch.randn_like(sigma)           # Sample noise (no learnable params)
z_reparam = mu + sigma * eps            # Deterministic function of mu, sigma

loss_reparam = z_reparam.sum()
loss_reparam.backward()

print("=== Reparameterization trick ===")
print(f"z = {z_reparam.detach().numpy()}")
print(f"mu.grad = {mu.grad}")
print(f"logvar.grad = {logvar.grad}")
print(f"Gradient flows to mu? {mu.grad is not None and mu.grad.abs().sum() > 0}")
print(f"Gradient flows to logvar? {logvar.grad is not None and logvar.grad.abs().sum() > 0}")
print()
print("The trick: isolate randomness in eps (not learnable),")
print("compute z = mu + sigma * eps (deterministic in mu, sigma).")
print("Gradients flow through mu and sigma normally.")

**What just happened:**

- Direct sampling (`dist.sample()`) creates a value that has no gradient connection to the distribution parameters. Calling `.backward()` gives `mu.grad = None` — the gradient path is broken.
- The reparameterization trick reformulates sampling as `z = mu + sigma * eps`. Since `eps` is just a constant (from PyTorch's perspective), `z` is a differentiable function of `mu` and `sigma`. Gradients flow.

This is the clever engineering trick that makes VAEs trainable. Without it, we couldn't backpropagate through the sampling step.

---

## Exercise 2: Convert the Autoencoder to a VAE (Guided)

Now we convert the autoencoder into a VAE. There are exactly **three changes**:

1. The encoder outputs `mu` and `logvar` instead of a single point `z`
2. The `reparameterize()` method samples `z` from the distribution
3. `forward()` returns `recon, mu, logvar` (the loss function needs mu and logvar)

Everything else — the conv layers, the decoder — is identical to the autoencoder.

**Before running, predict:** The autoencoder's encoder ends with `nn.Linear(32*7*7, latent_dim)` — one linear layer producing one vector. What does the VAE's encoder need instead?

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()

        # Encoder: same CNN backbone as the autoencoder
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        # CHANGE 1: Two output heads instead of one
        # The autoencoder had: nn.Linear(32*7*7, latent_dim) -> one point
        # The VAE has: two separate linear layers -> mu and logvar
        self.fc_mu = nn.Linear(32 * 7 * 7, latent_dim)
        self.fc_logvar = nn.Linear(32 * 7 * 7, latent_dim)

        # Decoder: identical to the autoencoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32 * 7 * 7),
            nn.ReLU(),
            nn.Unflatten(1, (32, 7, 7)),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    # CHANGE 2: Reparameterization trick (from Exercise 1)
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)    # sigma = exp(0.5 * log(sigma^2))
        eps = torch.randn_like(std)       # epsilon ~ N(0, 1)
        return mu + std * eps             # z = mu + sigma * epsilon

    def forward(self, x):
        # Encode to distribution parameters
        h = self.encoder_conv(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)

        # Sample z using reparameterization trick
        z = self.reparameterize(mu, logvar)

        # Decode
        recon = self.decoder(z)

        # CHANGE 3: Return mu and logvar too (the loss needs them)
        return recon, mu, logvar

    def decode(self, z):
        """Decode a latent vector without encoding first. For generation."""
        return self.decoder(z)


# Verify the architecture
vae_model = VAE(LATENT_DIM).to(device)

# Test with a single batch
test_images, _ = next(iter(test_loader))
test_images = test_images.to(device)
recon, mu, logvar = vae_model(test_images)

print(f"Input shape:   {test_images.shape}")
print(f"Recon shape:   {recon.shape}")
print(f"mu shape:      {mu.shape}")
print(f"logvar shape:  {logvar.shape}")
print()
print(f"mu range:      [{mu.min().item():.3f}, {mu.max().item():.3f}]")
print(f"logvar range:  [{logvar.min().item():.3f}, {logvar.max().item():.3f}]")
print()
print("The encoder now outputs a cloud (mu + logvar) for each image,")
print("not a single point. Each image is described by a distribution,")
print("not a location.")

**What just happened:**

The autoencoder's encoder ended with one linear layer: `image -> z` (a point). The VAE's encoder ends with **two** linear layers: `image -> mu` and `image -> logvar` (a distribution). The reparameterization trick then samples a `z` from that distribution.

Notice that `mu` and `logvar` each have shape `(batch_size, 2)` — each image gets its own 2D distribution. This is **per-image**, not one global distribution for the whole dataset. Each image becomes its own cloud in latent space.

---

## Exercise 3: VAE Loss and Training (Supported)

The VAE loss has two terms that **compete**:

$$\mathcal{L}_{\text{VAE}} = \underbrace{\mathcal{L}_{\text{recon}}}_{\text{reconstruction}} + \underbrace{\text{KL}(\,q(z|x)\;\|\;\mathcal{N}(0,1)\,)}_{\text{regularization}}$$

- **Reconstruction loss** wants sharp, specialized latent codes (same as the autoencoder)
- **KL divergence** wants organized, overlapping distributions near the center

The KL formula for Gaussian distributions is:

$$\text{KL} = -\frac{1}{2}\sum_{j=1}^{d}\left(1 + \log \sigma_j^2 - \mu_j^2 - \sigma_j^2\right)$$

In code: `kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())`

**Your task:** Fill in the TODO sections to implement the VAE loss function and training loop.

<details>
<summary>💡 Solution</summary>

The key insight is that the two loss terms serve different purposes. Reconstruction loss ensures the model can faithfully reproduce inputs (same as an autoencoder). KL divergence acts as a regularizer on the latent space shape — it penalizes means far from zero (don't hide in a corner) and penalizes small variances (don't collapse clouds to points). Together, they create a smooth, sampleable latent space.

```python
# TODO 1: KL divergence
kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# TODO 2: Total loss
loss = recon_loss + kl_loss
```

We use `reduction='sum'` for reconstruction loss so it matches the scale of the KL term, which is also a sum. Both are then normalized by batch size for stable training.

</details>

In [ ]:
def train_vae(model, train_loader, num_epochs=10, lr=1e-3):
    """Train a VAE with reconstruction + KL divergence loss."""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()

    history = {'total': [], 'recon': [], 'kl': []}

    for epoch in range(num_epochs):
        epoch_total, epoch_recon, epoch_kl = 0, 0, 0

        for images, _ in train_loader:
            images = images.to(device)
            recon, mu, logvar = model(images)

            # Reconstruction loss (same as autoencoder)
            recon_loss = F.mse_loss(recon, images, reduction='sum')

            # TODO 1: KL divergence loss
            # Formula: -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
            # This measures how far each image's distribution is from N(0, 1)
            kl_loss = ...  # YOUR CODE HERE (one line)

            # TODO 2: Total loss = reconstruction + KL
            # Normalize by batch size for stable training
            loss = ...  # YOUR CODE HERE (one line)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_size = images.size(0)
            epoch_total += loss.item() * batch_size
            epoch_recon += recon_loss.item()
            epoch_kl += kl_loss.item()

        n = len(train_loader.dataset)
        history['total'].append(epoch_total / n)
        history['recon'].append(epoch_recon / n)
        history['kl'].append(epoch_kl / n)

        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{num_epochs} | "
                  f"Total: {history['total'][-1]:.4f} | "
                  f"Recon: {history['recon'][-1]:.4f} | "
                  f"KL: {history['kl'][-1]:.4f}")

    return history


# Train the VAE
print("Training VAE...")
vae_model = VAE(LATENT_DIM).to(device)
vae_history = train_vae(vae_model, train_loader, num_epochs=20)
print("Done.")

In [ ]:
# Plot the training curves — watch the two losses compete
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(vae_history['total'], linewidth=2)
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')
axes[0].grid(alpha=0.3)

axes[1].plot(vae_history['recon'], linewidth=2, color='#4a9eff', label='Reconstruction')
axes[1].set_title('Reconstruction Loss')
axes[1].set_xlabel('Epoch')
axes[1].grid(alpha=0.3)

axes[2].plot(vae_history['kl'], linewidth=2, color='#ff6b6b', label='KL Divergence')
axes[2].set_title('KL Divergence')
axes[2].set_xlabel('Epoch')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice: Reconstruction loss decreases (model learns to reconstruct).")
print("KL divergence may increase early on as the encoder starts forming")
print("meaningful distributions, then stabilizes. The two losses compete.")

**What just happened:**

You trained a VAE with the two-term loss. The reconstruction loss pulls the model toward sharp, specialized latent codes (just like an autoencoder). The KL term pulls the other way — it penalizes means far from zero and variances that are too small. The tension between these two forces creates a latent space that is both useful (can reconstruct) and organized (can sample from).

Watch the KL curve: it typically rises at first (the encoder starts producing non-trivial distributions) then settles. If KL stays near zero, the model is collapsing to an autoencoder. If KL dominates, everything will look blurry.

---

## Exercise 4: Compare Latent Spaces (Supported)

The whole point of KL regularization is to fill the gaps in the latent space. Let's see if it worked. We'll encode the test set with both models and plot their 2D latent spaces side by side.

**Your task:** Fill in the TODO to extract the VAE's latent codes (mu values) from the test set. Then compare the two latent spaces.

<details>
<summary>💡 Solution</summary>

For the VAE, the encoder returns `(recon, mu, logvar)`. We want just `mu` — the center of each image's cloud. We collect these across all batches, just like we collect `z` from the autoencoder.

```python
# TODO: Get VAE latent codes
recon, mu, logvar = vae_model(images)
vae_codes.append(mu.detach().cpu())
```

We use `mu` (not a sample from the distribution) because `mu` is the center of each cloud — it gives the clearest picture of the latent space structure. Sampling would add noise.

</details>

In [ ]:
# Encode the entire test set with both models
ae_model.eval()
vae_model.eval()

ae_codes = []
vae_codes = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        # Autoencoder: encoder gives us z directly
        ae_z = ae_model.encoder(images)
        ae_codes.append(ae_z.cpu())

        # TODO: Get VAE latent codes (the mu values)
        # Hint: vae_model returns (recon, mu, logvar)
        # We want mu — the center of each image's cloud
        ...  # YOUR CODE HERE (2 lines: call vae_model, append mu)

        all_labels.append(labels)

ae_codes = torch.cat(ae_codes).numpy()
vae_codes = torch.cat(vae_codes).numpy()
all_labels = torch.cat(all_labels).numpy()

print(f"Encoded {len(all_labels)} test images.")
print(f"AE codes shape: {ae_codes.shape}")
print(f"VAE codes shape: {vae_codes.shape}")

In [ ]:
# Plot both latent spaces side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

colors = plt.cm.tab10(np.linspace(0, 1, 10))

for i in range(10):
    mask = all_labels == i
    axes[0].scatter(ae_codes[mask, 0], ae_codes[mask, 1],
                    c=[colors[i]], s=1, alpha=0.5, label=CLASS_NAMES[i])
    axes[1].scatter(vae_codes[mask, 0], vae_codes[mask, 1],
                    c=[colors[i]], s=1, alpha=0.5, label=CLASS_NAMES[i])

axes[0].set_title('Autoencoder Latent Space', fontsize=14)
axes[0].set_xlabel('z[0]')
axes[0].set_ylabel('z[1]')
axes[0].grid(alpha=0.2)

axes[1].set_title('VAE Latent Space (mu)', fontsize=14)
axes[1].set_xlabel('z[0]')
axes[1].set_ylabel('z[1]')
axes[1].grid(alpha=0.2)
axes[1].legend(loc='upper right', fontsize=8, markerscale=5)

plt.tight_layout()
plt.show()

# Print statistics to quantify the difference
print(f"\nAutoencoder latent range: [{ae_codes.min():.2f}, {ae_codes.max():.2f}]")
print(f"VAE latent range:         [{vae_codes.min():.2f}, {vae_codes.max():.2f}]")
print(f"\nAE codes std:  {ae_codes.std():.3f}")
print(f"VAE codes std: {vae_codes.std():.3f}")
print(f"\nNotice how the VAE latent space is more compact and centered.")
print(f"KL regularization pulled everything toward N(0, 1) — no far-flung corners.")

In [ ]:
# Decode random points from both latent spaces to see the gap problem
fig, axes = plt.subplots(2, 8, figsize=(16, 4))

# Sample random z from N(0, 1) — the space we want to be able to sample from
torch.manual_seed(42)
z_random = torch.randn(8, LATENT_DIM).to(device)

with torch.no_grad():
    ae_generated = ae_model.decoder(z_random).cpu()
    vae_generated = vae_model.decode(z_random).cpu()

for i in range(8):
    axes[0, i].imshow(ae_generated[i, 0], cmap='gray')
    axes[0, i].axis('off')
    axes[1, i].imshow(vae_generated[i, 0], cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('AE', fontsize=12, rotation=0, labelpad=25)
axes[1, 0].set_ylabel('VAE', fontsize=12, rotation=0, labelpad=25)

fig.suptitle('Random z ~ N(0,1) decoded by each model', fontsize=14)
plt.tight_layout()
plt.show()

print("Top row: Autoencoder. Random z lands in gaps → garbage.")
print("Bottom row: VAE. Random z lands in meaningful space → recognizable items.")
print("\nSame decoder architecture. Same random z. Different training objective.")
print("The KL regularizer is what makes the difference.")

**What just happened:**

The latent space comparison shows exactly what KL regularization does:

- **Autoencoder:** Scattered points with arbitrary range. Clusters may be far apart with empty gaps between them. Random z from N(0,1) likely falls in a gap — the decoder produces garbage.
- **VAE:** Compact, centered distributions near the origin. The clouds overlap. Random z from N(0,1) lands in meaningful territory — the decoder produces recognizable items.

KL divergence is the regularizer that pulled everything toward N(0,1). It penalizes means far from zero ("don't hide in a corner") and variances too small ("don't collapse clouds to points"). The result: a smooth, continuous latent space with no gaps.

---

## Exercise 5: Generate New Digits (Independent)

You have a trained VAE with a smooth, sampleable latent space. Now use it to **generate** new Fashion-MNIST items that never existed in the training set.

**Your task:**
1. Sample a grid of z vectors from N(0, 1) — try a range spanning [-3, 3] on each axis
2. Decode each z vector into an image
3. Display the results as a grid that shows how the latent space varies smoothly

This is the generative payoff: sample any point from N(0, 1), decode it, get a plausible image. The autoencoder cannot do this. The VAE can, because KL regularization made the entire latent space meaningful.

<details>
<summary>💡 Solution</summary>

The key insight is that the VAE's latent space is smooth and continuous — you can create a grid of evenly spaced z values and decode each one. As you move through the grid, the decoded images should change smoothly. This is exactly the "roads between buildings" that the autoencoder lacked.

```python
# Create a grid of z values spanning the latent space
n = 15  # 15x15 grid
z1 = torch.linspace(-3, 3, n)
z2 = torch.linspace(-3, 3, n)

# Build all z vectors
grid_z = torch.zeros(n * n, LATENT_DIM)
idx = 0
for i in range(n):
    for j in range(n):
        grid_z[idx, 0] = z1[j]
        grid_z[idx, 1] = z2[n - 1 - i]  # flip so y increases upward
        idx += 1

# Decode all at once
vae_model.eval()
with torch.no_grad():
    generated = vae_model.decode(grid_z.to(device)).cpu()

# Stitch into one big image
canvas = np.zeros((n * 28, n * 28))
idx = 0
for i in range(n):
    for j in range(n):
        canvas[i*28:(i+1)*28, j*28:(j+1)*28] = generated[idx, 0].numpy()
        idx += 1

plt.figure(figsize=(10, 10))
plt.imshow(canvas, cmap='gray')
plt.title('VAE Latent Space Grid: z sampled from [-3, 3]', fontsize=14)
plt.xlabel('z[0]')
plt.ylabel('z[1]')
plt.xticks(np.linspace(0, n*28, 5), [f'{v:.1f}' for v in np.linspace(-3, 3, 5)])
plt.yticks(np.linspace(0, n*28, 5), [f'{v:.1f}' for v in np.linspace(3, -3, 5)])
plt.tight_layout()
plt.show()
```

Notice how the images change smoothly as you move across the grid. Similar items cluster together and blend into each other at the boundaries. There are no gaps — every point in the latent space decodes to something plausible.

</details>

In [ ]:
# YOUR CODE HERE
# 1. Create a grid of z values (e.g., 15x15) spanning [-3, 3] on each axis
# 2. Decode each z with vae_model.decode(z.to(device))
# 3. Display as a grid image
#
# This is the generative payoff — the autoencoder can't do this,
# but the VAE can because every point in the latent space is meaningful.


---

## Key Takeaways

1. **The reparameterization trick** lets gradients flow through sampling: sample epsilon from N(0,1), compute z = mu + sigma * epsilon. The randomness is in epsilon (not learnable), so gradients flow through mu and sigma normally.

2. **The VAE encodes to a distribution (mu + logvar), not a point.** Each image becomes a cloud in latent space. Nearby images have overlapping clouds, filling the gaps that made autoencoder generation fail.

3. **KL divergence is a regularizer on the latent space shape.** Two intuitions: don't hide codes in a corner (penalizes large means), and don't make clouds so small they're basically points (penalizes small variance). Same principle as L2 on weights — constraints force better representations.

4. **The VAE loss = reconstruction + KL, and the two terms compete.** Reconstruction wants sharp, specialized codes. KL wants organized, overlapping distributions. VAE reconstructions are blurrier than autoencoder reconstructions — that's the price of a smooth, sampleable latent space.

5. **The result: a smooth latent space you can sample from.** Sample any point from N(0,1), decode it, and you get a plausible image. The autoencoder's failure is fixed — you have built your first true generative model.